# 角度変化を描画

In [14]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('ggplot') 

import numpy as np
import seaborn as sns
import pandas as pd
import math 

In [15]:
motion_frame = {
    "data" : [[8,61],[290,335],[612,653]]
}

## 関数定義（３点のなす角を求める 0≦角度≦180）

In [16]:
import math

def get_degrees(x1,y1,x2,y2):
    try:
        dot = x1*x2 + y1*y2
        vlength = math.sqrt(pow(x1, 2)+pow(y1, 2))*math.sqrt(pow(x2, 2)+pow(y2, 2))
        rad = math.acos(dot/vlength)
        deg = math.degrees(rad)
    except:
        print('get_degrees_error_{}_{}_{}_{}'.format(x1,y1,x2,y2))
        deg = 0

    return deg

## 関数定義（角度を求める）

In [17]:
def get_3point_degrees(df_sec, pos1_pos, base_pos, pos2_pos):
    degrees = []
        
    base = df_sec.query('position == @base_pos')
    pos1 = df_sec.query('position == @pos1_pos')
    pos2 = df_sec.query('position == @pos2_pos') 

    for i in np.arange(len(base.index)):
        deg = get_degrees(pos1.iloc[i].x-base.iloc[i].x, pos1.iloc[i].y-base.iloc[i].y, pos2.iloc[i].x-base.iloc[i].x, pos2.iloc[i].y-base.iloc[i].y)
        degrees.append(deg)

    df_deg = pd.DataFrame({"y":degrees, "position":([base_pos] * len(degrees))})
    
    # 欠損値を埋める
    df_deg = df_deg.fillna(method='ffill')
       
    return df_deg
        
def get_2point_degrees(df_sec, base_pos, pos1_pos):
    degrees = []
        
    base = df_sec.query('position == @base_pos')
    pos1 = df_sec.query('position == @pos1_pos')

    for i in np.arange(len(base.index)):
        # X軸との角度を求める
        x = abs(pos1.iloc[i].x-base.iloc[i].x)
        y = abs(pos1.iloc[i].y-base.iloc[i].y)
        degrees.append(math.degrees(math.atan(y / x)))
    
    df_deg = pd.DataFrame({"y":degrees, "position":([pos1_pos] * len(degrees))})

    # 欠損値を埋める
    df_deg = df_deg.fillna(method='ffill')
       
    return df_deg

## 関数定義（折れ線グラフを描画する）

In [18]:
def line_chart_degree(df, playerName, frame_cnt):
    
    for start, end in frame_cnt:
        df_sec = df.query('@start <= frameNo & frameNo <= @end' )
        
        # 肩と腰
        plt.figure(figsize=(10,7))
        # 肩
        df_deg = get_2point_degrees(df_sec,2, 5)
        plt.plot(np.arange(len(df_deg.index)), df_deg["y"].as_matrix(), label="shoulder")
        
        # 腰
        df_deg = get_2point_degrees(df_sec, 8,11)
        plt.plot(np.arange(len(df_deg.index)), df_deg["y"].as_matrix(), label="hip")
        
        # 右肘
        df_deg = get_3point_degrees(df_sec, 2, 3, 4)
        plt.plot(np.arange(len(df_deg.index)), df_deg["y"].as_matrix(), label="elbow")
        
        plt.xlim([0, len(df_deg.index)])
        plt.ylim([-10, 200])
        plt.title('{}:{}-{}'.format(playerName, start, end), fontsize=20, fontname='serif') # タイトル
        plt.legend(loc='upper left')
        plt.savefig('./line_chart_{}_{}_{}.png'.format(playerName, start, end))
        plt.close()


## 折れ線グラフを作成

In [19]:
for playerName, frame_cnt in motion_frame.items():
    
    # 画像の座標データを取得する
    df = pd.read_csv('./motion_data.csv')
    
    # 折れ線グラフ
    line_chart_degree(df ,playerName, frame_cnt)    

C:\tools\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
C:\tools\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\tools\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


get_degrees_error_23_-108_-23_108
